# 모니터 추론하기

지금까지 이미지 및 동영상파일, 내장카메라 및 USB카메라, IP카메라 등 다양한 인풋에 대해 실습해보았습니다.

이번 챕터에서는 마지막으로 모니터를 인풋으로 추론을 수행하는 코드를 알아보겠습니다.

(참고로 이번 예제에서는 mss 모듈을 추가로 설치합니다. 이를 통해 스크린샷을 빠르게 찍습니다.)

기본적으로 주모니터의 인덱스는 0입니다. 보조모니터가 있다면 연결 순서대로 1, 2, 3이 됩니다.

아래 코드를 읽어본 후 실행해보시기 바랍니다.

이전과 마찬가지로 화면에 보이는 사람의 수를 세어서 화면에 출력하는 작업을 수행합니다.


In [2]:
import mss

In [ ]:
%pip install ultralytics opencv-python mss numpy

In [5]:
import os

from ultralytics import YOLO
import cv2
import mss  # 현존 최고의 스크린샷 모듈


def on_exists(fname: str) -> None:
    if os.path.isfile(fname):
        os.remove(fname)


model = YOLO("yolov8n.pt")
classNames = model.names

while True:
    with mss.mss() as sct:
        img = cv2.imread(sct.shot(output="monitor.png",
            mon=0,  # 보조(2번)모니터 찍기. mon 대신 bounding_box 사용 가능
            callback=on_exists))  # 기존에 파일이 있으면 지우고 나서 저장
        results = model(img)
        for r in results:
            boxes = r.boxes
            total_person_no = 0
            for box in boxes:
                if int(box.cls) == 0:  # 사람이면
                    # 박스 추출
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values

                    # 네모 그리기
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

                    # 신뢰도 추출
                    confidence = box.conf[0]

                    # 클래스명 추출
                    cls = int(box.cls[0])

                    if cls == 0:  # 사람이면
                        total_person_no += 1

                    # 주석 삽입용 정보
                    org = [x1, y1]
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 1
                    color = (255, 0, 0)
                    thickness = 1

                    cv2.putText(
                        img, classNames[cls], org, font, 
                        fontScale, color, thickness)
            print("total person no : ", total_person_no)

            cv2.imshow('screen', img)

    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        cv2.destroyAllWindows()
        break

0: 416x640 3 persons, 1443.7ms
Speed: 31.7ms preprocess, 1443.7ms inference, 16.1ms postprocess per image at shape (1, 3, 416, 640)
total person no :  3
0: 416x640 6 persons, 237.7ms
Speed: 6.5ms preprocess, 237.7ms inference, 4.0ms postprocess per image at shape (1, 3, 416, 640)
total person no :  6
0: 416x640 8 persons, 1 giraffe, 1 tv, 269.7ms
Speed: 7.0ms preprocess, 269.7ms inference, 4.5ms postprocess per image at shape (1, 3, 416, 640)
total person no :  8
0: 416x640 8 persons, 1 giraffe, 1 tv, 257.2ms
Speed: 6.5ms preprocess, 257.2ms inference, 5.0ms postprocess per image at shape (1, 3, 416, 640)
total person no :  8
0: 416x640 8 persons, 1 giraffe, 1 tv, 261.9ms
Speed: 7.0ms preprocess, 261.9ms inference, 4.3ms postprocess per image at shape (1, 3, 416, 640)
total person no :  8
0: 416x640 8 persons, 1 giraffe, 2 tvs, 254.1ms
Speed: 7.5ms preprocess, 254.1ms inference, 4.0ms postprocess per image at shape (1, 3, 416, 640)
total person no :  8
0: 416x640 10 persons, 1 giraffe,


촬영영역 안에 cv 창이 떠 있으면 화면이 복잡해지더라도

q를 누르면 종료되니 이 점 기억해 주시기 바랍니다.

아래 코드는 동일한 작업을 수행하되 모니터의 특정 영역에 대해서만 추론을 수행합니다.

영역은 top, left, width, height 등 네 개의 키로 정의하며,

아래 코드는 사분할 중 우측 하단 영역에 대해서만 추론을 수행합니다. (FHD1920x1080 기준)

(20번 라인부터는 100% 동일합니다.)


In [ ]:
import os

from ultralytics import YOLO
import cv2
import mss  # 현존 최고의 스크린샷 모듈
import numpy as np

bounding_box = {'top': 540, 'left': 960, 'width': 960, 'height': 540}

model = YOLO("yolov8n.pt")
classNames = model.names


while True:
    with mss.mss() as sct:
        img = np.array(sct.grab(bounding_box))[:, :, :3]  # rgba to rgb
        
        if not img.flags['C_CONTIGUOUS']:  # 연속메모리배열 여부인지 확인 및 변환
            img = np.ascontiguousarray(img)
        results = model(img)
        for r in results:
            boxes = r.boxes
            total_person_no = 0
            for box in boxes:
                if int(box.cls) == 0:  # 사람이면
                    # 박스 추출
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values

                    # 네모 그리기
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

                    # 신뢰도 추출
                    confidence = box.conf[0]

                    # 클래스명 추출
                    cls = int(box.cls[0])

                    if cls == 0:  # 사람이면
                        total_person_no += 1

                    # 주석 삽입용 정보
                    org = [x1, y1]
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 1
                    color = (255, 0, 0)
                    thickness = 1

                    cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)
            print("total person no : ", total_person_no)

            cv2.imshow('screen', img)

    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        cv2.destroyAllWindows()
        break



여기까지 모니터 영역을 입력으로 추론을 수행하는 작업을 해보았습니다.

캡쳐나 다운로드가 불가능하지만 웹브라우저에서 조회는 가능한 경우에

특히 유용한 추론방법이므로, 기억해 두셨다가 활용해 주시기 바랍니다.

![](https://i.ibb.co/xHk8WfL/2024-02-01-16-47-59-59.gif)

